In [2]:
import pandas as pd
import numpy as np
import random as rnd

# visualization
#import seaborn as sns
import matplotlib.pyplot as plt
#%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
#train = pd.read_csv("Train File Name")
train = pd.read_csv("final.csv")


In [6]:
#test = pd.read_csv("Test File Name")
test = pd.read_csv("final2.csv")


In [8]:

train=train[train['RIO_ACCEPT_REJECT']!='SM']
test=test[test['RIO_ACCEPT_REJECT']!='SM']


In [10]:
train['RIO_ACCEPT_REJECT']= train['RIO_ACCEPT_REJECT'].map({'NM': 0, 'M' : 1})
test['RIO_ACCEPT_REJECT']= test['RIO_ACCEPT_REJECT'].map({'NM': 0, 'M' : 1})

In [12]:
def counter(x):
    c=0
    y=x.split(',')
    #print(x)
    for i in y:
        if(i!=' '):
            c=c+1
    return c

In [14]:
def reduction(x):
    if x==-1:
        return 0
    else:
        return x

In [16]:




def cut(prob,i):
    return [1 if x >=i else 0 for x in prob]

def testresults(prob,testx,testy,model_var,name):
    Y_pred=model_var.predict(testx)
    correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(Y_pred,testy)]
    print((sum(correct)/len(correct))*100)
    tp = [1 if ((a == 1 and b == 1)) else 0 for (a, b) in zip(Y_pred,testy)]
    fp = [1 if ((a == 1 and b == 0)) else 0 for (a, b) in zip(Y_pred,testy)]
    fn= [1 if ((a == 0 and b == 1)) else 0 for (a, b) in zip(Y_pred,testy)]
    tn= [1 if ((a == 0 and b == 0)) else 0 for (a, b) in zip(Y_pred,testy)]
    tp=sum(tp)
    fp=sum(fp)
    fn=sum(fn)
    tn=sum(tn)
    print("Model:"+str(model_var))
    print(tp,fp,fn,tn)
    P=(tp)/(tp+fp)
    R=(tp)/(tp+fn)
    print("Precision:"+str(P*100),"\tRecall:"+str(R*100))
    F=2*P*R
    F=F/(P+R)
    print("F1 SCORE:"+str(F*100))
    
    syn1=pd.DataFrame({"MODEL":name,"PRECISION":P*100,"RECALL":R*100,"F1_SCORE":F*100,"POS":(tp+fn),"NEG":(fp+tn),"TP":tp,"FP":fp,"FN":fn,"TN":tn},index=[0])
    syn=syn1[['MODEL','POS','NEG','F1_SCORE','PRECISION','RECALL','TP','FP','FN','TN']]
    return syn



In [18]:
train['MATCH_TYPE2']=train['MATCH_TYPE'].apply(counter)
test['MATCH_TYPE2']=test['MATCH_TYPE'].apply(counter)
train['COL1040STRENGTH']=train['COL1040STRENGTH'].apply(reduction)
test['COL1040STRENGTH']=test['COL1040STRENGTH'].apply(reduction)
train['COL1050STRENGTH']=train['COL1050STRENGTH'].apply(reduction)
test['COL1050STRENGTH']=test['COL1050STRENGTH'].apply(reduction)
train['COL2000STRENGTH']=train['COL2000STRENGTH'].apply(reduction)
test['COL2000STRENGTH']=test['COL2000STRENGTH'].apply(reduction)
train['COL1000LVL']=train['COL1000LVL'].apply(reduction)
test['COL1000LVL']=test['COL1000LVL'].apply(reduction)
train['ADDRMINLENSTRENGTH']=(train['ADDR1MINLENSTRENGTH']+train['ADDR2MINLENSTRENGTH']+train['ADDR3MINLENSTRENGTH'])/3
test['ADDRMINLENSTRENGTH']=(test['ADDR1MINLENSTRENGTH']+test['ADDR2MINLENSTRENGTH']+test['ADDR3MINLENSTRENGTH'])/3
train['ADDRMAXLENSTRENGTH']=(train['ADDR1MAXLENSTRENGTH']+train['ADDR2MAXLENSTRENGTH']+train['ADDR3MAXLENSTRENGTH'])/3
test['ADDRMAXLENSTRENGTH']=(test['ADDR1MAXLENSTRENGTH']+test['ADDR2MAXLENSTRENGTH']+test['ADDR3MAXLENSTRENGTH'])/3
train['ADDRESSMATCHSTRENGTH']=(test['ADDRESSMATCH1STRENGTH']+test['ADDRESSMATCH2STRENGTH']+test['ADDRESSMATCH3STRENGTH'])/3
test['ADDRESSMATCHSTRENGTH']=(test['ADDRESSMATCH1STRENGTH']+test['ADDRESSMATCH2STRENGTH']+test['ADDRESSMATCH3STRENGTH'])/3

train=train.dropna()
test=test.dropna()

In [20]:
X=train[['COL1050STRENGTH','COL5000STRENGTH','ADDRMAXLENSTRENGTH','SCALE_TYPE','ADDRMINLENSTRENGTH','ADDRESSMATCHSTRENGTH','COL4000STRENGTH','MATCH_TYPE2','COL2000STRENGTH','COL5040STRENGTH','COL1000STRENGTH'
,'CS1STRENGTH','CS2STRENGTH','COL1000LVL','COL3000STRENGTH','COL5030STRENGTH','COL1040STRENGTH','CS3STRENGTH']]
y=train['RIO_ACCEPT_REJECT']
#y = y.reshape(len(train), 1)
type(y)
X.shape,y.shape

((42058, 18), (42058,))

In [21]:
from imblearn.over_sampling import SMOTE

sm=SMOTE(ratio='auto',kind="regular")
X_res, y_res = sm.fit_sample(X,y)
xx=pd.DataFrame(X_res)
yy=pd.DataFrame(y_res)



C:\Users\Hp_Owner\Anaconda3\lib\site-packages\pandas\core\frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\Hp_Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hp_Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [ ]:
sm1=xx[yy[0]==1]
sm1[['COL1050STRENGTH','COL5000STRENGTH','ADDRMAXLENSTRENGTH','SCALE_TYPE','ADDRMINLENSTRENGTH','ADDRESSMATCHSTRENGTH','COL4000STRENGTH','MATCH_TYPE2','COL2000STRENGTH','COL5040STRENGTH','COL1000STRENGTH'
,'CS1STRENGTH','CS2STRENGTH','COL1000LVL','COL3000STRENGTH','COL5030STRENGTH','COL1040STRENGTH','CS3STRENGTH']]=xx[yy[0]==1]
smfin=sm1[['COL1050STRENGTH','COL5000STRENGTH','ADDRMAXLENSTRENGTH','SCALE_TYPE','ADDRMINLENSTRENGTH','ADDRESSMATCHSTRENGTH','COL4000STRENGTH','MATCH_TYPE2','COL2000STRENGTH','COL5040STRENGTH','COL1000STRENGTH'
,'CS1STRENGTH','CS2STRENGTH','COL1000LVL','COL3000STRENGTH','COL5030STRENGTH','COL1040STRENGTH','CS3STRENGTH']]

smy=yy[yy[0]==1]
smy['RIO_ACCEPT_REJECT']=smy
smyfin=smy['RIO_ACCEPT_REJECT']

framesx=[X,smfin[:30000]]
framesy=[y,smyfin[:30000]]
X=pd.concat(framesx)
y=pd.concat(framesy)
y = y.reshape(len(X), 1)

In [22]:

input_layer_size=X.shape[1]
hidden_layer_size=6
num_labels=2

counter=0

def sigmoid(z):  
    return 1 / (1 + np.exp(-z))


# In[45]:

def sigmoidGradient(z):
    g=np.matrix(np.zeros(z.shape))
    g=np.multiply(sigmoid(z),(1-sigmoid(z)))
    return g
        
    


# In[46]:

def forwardpropagate(X,theta1,theta2):

   # theta1 = np.matrix(np.reshape(params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
    #theta2 = np.matrix(np.reshape(params[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))
    a1=np.matrix(np.insert(X,0,1,axis=1))
    #print("a1:\t"+str(a1.shape))
    z2=np.matmul(a1,theta1.T)
    a2=sigmoid(z2)
    a2=np.insert(a2,0,1,axis=1)
    #print("a2:\t"+str(a2.shape))
    z3=np.matmul(a2,theta2.T)
    a3=sigmoid(z3)
    return a3



# In[47]:

def nnCostRegGrad(params,input_layer_size,hidden_layer_size,num_labels,X,y,lam):
    theta1 = np.matrix(np.reshape(params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
    theta2 = np.matrix(np.reshape(params[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))
    global counter
    #print(theta1.shape)
    #print(theta2.shape)
    m=X.shape[0]
    X=np.matrix(X)
    a1=np.matrix(np.insert(X,0,1,axis=1))
    #print("a1:\t"+str(a1.shape))
    z2=np.matmul(a1,theta1.T)
    a2=sigmoid(z2)
    a2=np.insert(a2,0,1,axis=1)
    #print("a2:\t"+str(a2.shape))
    z3=np.matmul(a2,theta2.T)
    a3=sigmoid(z3)
  
                       
    y2=np.copy(y)        
    
    #print("y2:\t"+str(y2.shape))
    y2=np.matrix(y2)  
    a3=np.matrix(a3)
    a2=np.matrix(a2)
    a1=np.matrix(a1)
    theta1=np.matrix(theta1)
    theta2=np.matrix(theta2)
    #print(y2)
    M=np.multiply(-y2.T,np.log(a3))-np.multiply((1-y2.T),np.log(1-a3))
    l=np.sum(np.sum(M))
    J=l/m
    
    t1=np.power(theta1,2)
    t2=np.power(theta2,2)
    reg1=np.sum(np.sum(t1[:,2:theta1.shape[1]]))
    reg2=np.sum(np.sum(t2[:,2:theta1.shape[1]]))
    
    reg=reg1+reg2
    
    finreg=(lam*reg)/(2*m)
    J+=finreg
    
    
    
    #print(J)
    yy=np.matrix(y2.T)
    b=np.matrix(np.insert(X,0,1,axis=1))
    findel1=np.matrix(np.zeros(theta1.shape))
    findel2=np.matrix(np.zeros(theta2.shape))
    #print(findel1.shape)
    #print(findel2.shape)
    percentage=100*(counter/250)
    print(percentage)
    counter=counter+1
    for t in range(X.shape[0]):
        aa1=np.matrix(b[t])
        
        zz2=np.matmul(aa1,theta1.T)
        aa2=sigmoid(zz2)
        aa2=np.insert(aa2,0,1,axis=1)
        
        #print("aa2:\t"+str(aa2.shape))
        zz3=np.matmul(aa2,theta2.T)
        aa3=sigmoid(zz3)
        #print(aa3)
        #print(yy[t].shape)
        d3=aa3-yy[t]
        #print("d3:\t"+str(d3.shape))
        d2=np.multiply(np.matmul(d3,theta2),sigmoidGradient(np.insert(zz2,0,1)))
        #print("d2:\t"+str(d2.shape))
        #print(findel1.shape)
        #print("d2 before:\t"+str(d2.shape))
        d2=d2[:,1:]
     #   print("d2:\t"+str(d2.shape))
        findel1=findel1+np.matmul(d2.T,aa1)
        #print(findel1)
        findel2=findel2+np.matmul(d3.T,aa2)
        #print(findel2)
    Theta1_grad=np.matrix(np.zeros(theta1.shape))
    Theta2_grad=np.matrix(np.zeros(theta2.shape))
    Theta1_grad=findel1/m
    Theta2_grad=findel2/m
    
    #print(Theta1_grad.shape,Theta2_grad.shape)
    
    Theta1_grad[:,1:Theta1_grad.shape[1]]=Theta1_grad[:,1:]+((lam*theta1[:,1:]))/m
    Theta2_grad[:,1:Theta2_grad.shape[1]]=Theta2_grad[:,1:]+((lam*theta2[:,1:]))/m
    
    grad = np.concatenate((np.ravel(Theta1_grad), np.ravel(Theta2_grad)))
    
    return J,grad




In [23]:
y2=np.zeros((num_labels,X.shape[0]))

    
for i in range(0,X.shape[0]):
    for j in range(0,num_labels):
        if(y[i]==j):
            y2[j][i]=1

params = (np.random.random(size=hidden_layer_size * (input_layer_size + 1) + num_labels * (hidden_layer_size + 1)) - 0.5) * 0.25

# unravel the parameter array into parameter matrices for each layer
J,grad=nnCostRegGrad(params,input_layer_size,hidden_layer_size,num_labels,X,y2,lam=1)
y


# In[49]:



0.0


array([[0],
       [0],
       [0],
       ..., 
       [1],
       [1],
       [1]], dtype=int64)

In [24]:
from scipy.optimize import minimize
lam=1
# minimize the objective function
fmin = minimize(fun=nnCostRegGrad, x0=params, args=(input_layer_size, hidden_layer_size, num_labels, X, y2,lam),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin


# In[50]:

X = np.matrix(X)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))

h = forwardpropagate(X,theta1,theta2)  
y_pred = np.array(np.argmax(h, axis=1))  
y,y_pred


# In[51]:

correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print ('accuracy = {0}%'.format(accuracy * 100))

0.4
0.8
1.2
1.6
2.0
2.4
2.8000000000000003
3.2
3.5999999999999996
4.0
4.3999999999999995
4.8
5.2
5.6000000000000005
6.0
6.4
6.800000000000001
7.199999999999999
7.6
8.0
8.4
8.799999999999999
9.2
9.6
10.0
10.4
10.8
11.200000000000001
11.600000000000001
12.0
12.4
12.8
13.200000000000001
13.600000000000001
14.000000000000002
14.399999999999999
14.799999999999999
15.2
15.6
16.0
16.400000000000002
16.8
17.2
17.599999999999998
18.0
18.4
18.8
19.2
19.6
20.0
20.4
20.8
21.2
21.6
22.0
22.400000000000002
22.8
23.200000000000003
23.599999999999998
24.0
24.4
24.8
25.2
25.6
26.0
26.400000000000002
26.8
27.200000000000003
27.6
28.000000000000004
28.4
28.799999999999997
29.2
29.599999999999998
30.0
30.4
30.8
31.2
31.6
32.0
32.4
32.800000000000004
33.2
33.6
34.0
34.4
34.8
35.199999999999996
35.6
36.0
36.4
36.8
37.2
37.6
38.0
38.4
38.800000000000004
39.2
39.6
40.0
40.400000000000006
40.8
41.199999999999996
41.6
42.0
42.4
42.8
43.2
43.6
44.0
44.4
44.800000000000004
45.2
45.6
46.0
46.400000000000006
46.800

C:\Users\Hp_Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: overflow encountered in exp


50.4
50.8
51.2
51.6
52.0
52.400000000000006
52.800000000000004
53.2
53.6
54.0
54.400000000000006
54.800000000000004
55.2
55.60000000000001
56.00000000000001
56.39999999999999
56.8
57.199999999999996
57.599999999999994
57.99999999999999
58.4
58.8
59.199999999999996
59.599999999999994
60.0
60.4
60.8
61.199999999999996
61.6
62.0
62.4
62.8
63.2
63.6
64.0
64.4
64.8
65.2
65.60000000000001
66.0
66.4
66.8
67.2
67.60000000000001
68.0
68.4
68.8
69.19999999999999
69.6
70.0
70.39999999999999
70.8
71.2
71.6
72.0
72.39999999999999
72.8
73.2
73.6
74.0
74.4
74.8
75.2
75.6
76.0
76.4
76.8
77.2
77.60000000000001
78.0
78.4
78.8
79.2
79.60000000000001
80.0
80.4
80.80000000000001
81.2
81.6
82.0
82.39999999999999
82.8
83.2
83.6
84.0
84.39999999999999
84.8
85.2
85.6
86.0
86.4
86.8
87.2
87.6
88.0
88.4
88.8
89.2
89.60000000000001
90.0
90.4
90.8
91.2
91.60000000000001
92.0
92.4
92.80000000000001
93.2
93.60000000000001
94.0
94.39999999999999
94.8
95.19999999999999
95.6
96.0
96.39999999999999
96.8
97.2
97.6
98.0
9

In [25]:
#data2=data2.dropna()
#fnx2=np.array(data2[['COL5000STRENGTH','COL4000STRENGTH','MATCH_TYPE2','COL2000STRENGTH','COL5040STRENGTH','COL1000STRENGTH','CS1STRENGTH','CS2STRENGTH','COL1000LVL','COL3000STRENGTH','COL5030STRENGTH','CS3STRENGTH']])
#testy=data2['RIO_ACCEPT_REJECT']
#len(data2)

In [26]:
test=test.dropna()
fnx2=np.array(test[['COL1050STRENGTH','COL5000STRENGTH','ADDRMAXLENSTRENGTH','SCALE_TYPE','ADDRMINLENSTRENGTH','ADDRESSMATCHSTRENGTH','COL4000STRENGTH','MATCH_TYPE2','COL2000STRENGTH','COL5040STRENGTH','COL1000STRENGTH','CS1STRENGTH','CS2STRENGTH','COL1000LVL','COL3000STRENGTH','COL5030STRENGTH','COL1040STRENGTH','CS3STRENGTH']])
testy=test['RIO_ACCEPT_REJECT']


In [27]:

h2 = forwardpropagate(fnx2,theta1,theta2)  
y_pred2 = np.array(np.argmax(h2, axis=1))  
#y2,y_pred2


In [28]:
fny2 = np.array(test['RIO_ACCEPT_REJECT'])

In [29]:
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(y_pred2,fny2)]

In [31]:
tp = [1 if ((a == 1 and b == 1)) else 0 for (a, b) in zip(y_pred2,fny2)]  
fp = [1 if ((a == 1 and b == 0)) else 0 for (a, b) in zip(y_pred2,fny2)]  
fn= [1 if ((a == 0 and b == 1)) else 0 for (a, b) in zip(y_pred2,fny2)]  
sum(tp)
tp=sum(tp)
fp=sum(fp)
fn=sum(fn)
P=(tp)/(tp+fp)
R=(tp)/(tp+fn)
F=2*P*R
F=F/(P+R)
F,P,R

(0.36001101624896725, 0.23249857712009106, 0.797267626250305)

In [39]:
test['MODEL_PREDICTION']=y_pred2
#Write the name of the output file
test.to_csv("NeuralNetwork.csv")